In [1]:
from __future__ import print_function
import time
import numpy as np
np.random.seed(1337)  # for reproducibility

from keras.preprocessing import sequence
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM, Input, merge, BatchNormalization,GRU
from keras.datasets import imdb

import os
from keras.preprocessing.text import Tokenizer

Using Theano backend.


In [2]:
X_train = []
y_train = []

path = 'C:/Data_Eng/test_remove_stopword/train/pos/'
X_train.extend([open(path + f).read() for f in os.listdir(path) if f.endswith('.txt')])

y_train.extend([1 for _ in range(13)])

path = 'C:/Data_Eng/test_remove_stopword/train/neg/'
X_train.extend([open(path + f).read() for f in os.listdir(path) if f.endswith('.txt')])

y_train.extend([0 for _ in range(11)])


In [3]:

X_test = []
y_test = []

path = 'C:/Data_Eng/test_remove_stopword/test/pos/'
X_test.extend([open(path + f).read() for f in os.listdir(path) if f.endswith('.txt')])
y_test.extend([1 for _ in range(13)])


path = 'C:/Data_Eng/test_remove_stopword/test/neg/'
X_test.extend([open(path + f).read() for f in os.listdir(path) if f.endswith('.txt')])
y_test.extend([0 for _ in range(15)])

In [4]:

max_features =5000
max_len = 200  # cut texts after this number of words (among top max_features most common words)

In [7]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

X_train_rm= []
stop_words=set(stopwords.words("english"))
for x in X_train:
    words=word_tokenize(x)
    remove_sw= [w for w in words if not w in stop_words]
    X_train_rm.append(remove_sw),
sentence_train=[] 
for i in range(len(X_train_rm)):
    s = ""
    for j in range(len(X_train_rm[i])):
        s+=X_train_rm[i][j]+" "
    sentence_train.append(s),
print(sentence_train),
        

 
        
#tokenize works to list of integers where each integer is a key to a word
#imdbTokenizer = Tokenizer(nb_words=max_features)

#imdbTokenizer.fit_on_texts(X_train)




['inco net soars higher metal prices breakup fee dale crofts z canadian nickel producer bought brazil cia vale rio said quarter profit soared fold boosted surging fees paid falconbridge failed takeover income jumped million share cents year earlier toronto based today statement results included deals phelps dodge corp sales billion sold double price average output percent demand used stainless steel surged global economic growth fueled especially china mines pace prompting buying spree producers seeking bolster ore deposits outbid teck cominco bid record quarterly earnings reflect unprecedented sustained strength seen market combined strong production chief executive officer scott hand shares gained c p stock exchange past janeiro world largest iron offered got merger payments lionore mining international strike workers voisey bay september led charge biggest copper dropped offer cash shareholders supported unsolicited climbed pounds metric ton company cut forecast strikes equipment fa

(None,)

In [8]:
X_test_rm= []
stop_words=set(stopwords.words("english"))
for x in X_test:
    words=word_tokenize(x)
    remove_sw= [w for w in words if not w in stop_words]
    X_test_rm.append(remove_sw),
sentence_test=[] 
for i in range(len(X_test_rm)):
    s = ""
    for j in range(len(X_test_rm[i])):
        s+=X_test_rm[i][j]+" "
    sentence_test.append(s),
print(sentence_test),

['obama sings blues white house event b king r g e l v z president barack sang prompting genre best known musicians gathered concert night mark black history month mick jagger buddy guy jeff beck artists taking stage eighth installment performance series thrill gone offered turn loose selections song set said roots slavery segregation indignities faced past americans refused limited spread south chicago world lay foundation rock roll hip hop universal appeal goes life joy pain triumph sorrow reminds tougher times crossroads shy away problems face deal sing plans speak today construction site smithsonian national museum african american culture open break rehearsals day met families talking music politics keb mo troy trombone shorty andrews spoke reporters yesterday told fans message feb social networking twitter really relaxed happy goaded singing words sweet home reference hometown ensemble finished heard al green teased started audience demurred took microphone crowd delight previous

(None,)

In [9]:
#tokenize works to list of integers where each integer is a key to a word
imdbTokenizer = Tokenizer(nb_words=max_features)

imdbTokenizer.fit_on_texts(sentence_train)
for word, value in imdbTokenizer.word_index.items():
    print(word),


limited
ccarpenter
plenary
asian
consists
appetite
forget
increase
aug
paris
regan
worth
lure
risk
sca
regional
sellers
jack
affect
bringing
companies
herbert
yongshun
burke
force
specially
guidelines
surrounding
second
estimated
machines
even
established
hide
qaddafi
asia
spokesman
bonuses
falconbridge
supplies
net
seeks
told
million
reporter
atoms
reported
protection
china
represented
digits
items
k
climbed
postpones
reports
electricity
analysts
military
postponed
changes
issuers
myasnikovich
highly
moral
total
unit
opponents
derivatives
would
smelter
negative
distributing
asset
strike
assessment
tolerated
composite
benefiting
award
aware
phone
connecticut
allies
rights
pursue
work
plunged
changsha
era
install
henry
sets
example
lejiang
stagnated
estate
give
cited
india
syvazinvest
currency
crofts
times
returned
end
recovery
thing
provide
recovers
machine
hot
interview
rise
earlier
hong
customs
president
ludlum
goaltenders
effective
micex
amid
schneider
supermarkets
greek
complexity


In [10]:
#create int to word dictionary
intToWord = {}
for word, value in imdbTokenizer.word_index.items():
    intToWord[value] = word

#add a symbol for null placeholder
intToWord[0] = "!!!NA!!!"
    
print(intToWord[1])
print(intToWord[2])
print(intToWord[32])

said
company
growth


In [11]:
#convert word strings to integer sequence lists
#print(X_train[0])
#print(imdbTokenizer.texts_to_sequences(X_train[:1]))
#for value in imdbTokenizer.texts_to_sequences(X_train[:1])[0]:
    #print(intToWord[value])
    
X_train = imdbTokenizer.texts_to_sequences(sentence_train)
X_test = imdbTokenizer.texts_to_sequences(sentence_test)

In [12]:

# Censor the data by having a max review length (in number of words)

#use this function to load data from keras pickle instead of munging as shown above
#(X_train, y_train), (X_test, y_test) = imdb.load_data(nb_words=max_features,
#                                                      test_split=0.2)

print(len(X_train), 'train sequences')
print(len(X_test), 'test sequences')

print("Pad sequences (samples x time)")
X_train = sequence.pad_sequences(X_train, maxlen=max_len)
X_test = sequence.pad_sequences(X_test, maxlen=max_len)
print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)
y_train = np.array(y_train)
y_test = np.array(y_test)

24 train sequences
28 test sequences
Pad sequences (samples x time)
X_train shape: (24L, 200L)
X_test shape: (28L, 200L)


In [13]:
epochs = 2
embedding_neurons = 128
lstm_neurons = 64
batch_size =32

In [14]:

# Forward Pass LSTM Network

# this is the placeholder tensor for the input sequences
sequence = Input(shape=(max_len,), dtype='int32')
# this embedding layer will transform the sequences of integers
# into vectors of size embedding
# embedding layer converts dense int input to one-hot in real time to save memory
embedded = Embedding(max_features, embedding_neurons, input_length=max_len)(sequence)
# normalize embeddings by input/word in sentence
bnorm = BatchNormalization()(embedded)

# apply forwards LSTM layer size lstm_neurons
forwards = GRU(lstm_neurons, dropout_W=0.4, dropout_U=0.4)(bnorm)

# dropout 
after_dp = Dropout(0.5)(forwards)
output = Dense(1, activation='sigmoid')(after_dp)

model_fdir_atom = Model(input=sequence, output=output)
# review model structure
print(model_fdir_atom.summary())

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 200)           0                                            
____________________________________________________________________________________________________
embedding_1 (Embedding)          (None, 200, 128)      640000      input_1[0][0]                    
____________________________________________________________________________________________________
batchnormalization_1 (BatchNorma (None, 200, 128)      512         embedding_1[0][0]                
____________________________________________________________________________________________________
gru_1 (GRU)                      (None, 64)            37056       batchnormalization_1[0][0]       
___________________________________________________________________________________________

In [15]:
# Forward pass LSTM network

# try using different optimizers and different optimizer configs
model_fdir_atom.compile('adam', 'binary_crossentropy', metrics=['accuracy','precision', 'recall', 'fmeasure'])
print('Train...')
start_time = time.time()

history_fdir_atom = model_fdir_atom.fit(X_train, y_train,
                    batch_size=batch_size,
                    nb_epoch=epochs,
                    validation_data=[X_test, y_test], 
                    verbose=2)

end_time = time.time()
average_time_per_epoch = (end_time - start_time) / epochs
print("avg sec per epoch:", average_time_per_epoch)

scores = model_fdir_atom.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Train...
Train on 24 samples, validate on 28 samples
Epoch 1/2
0s - loss: 0.8647 - acc: 0.4583 - precision: 0.5000 - recall: 0.4615 - fmeasure: 0.4800 - val_loss: 0.6948 - val_acc: 0.5000 - val_precision: 0.4800 - val_recall: 0.9231 - val_fmeasure: 0.6316
Epoch 2/2
0s - loss: 0.7182 - acc: 0.4583 - precision: 0.5000 - recall: 0.5385 - fmeasure: 0.5185 - val_loss: 0.6954 - val_acc: 0.5000 - val_precision: 0.4800 - val_recall: 0.9231 - val_fmeasure: 0.6316
avg sec per epoch: 6.64850008488
Accuracy: 50.00%


In [16]:

# Bi-directional Atom

# based on keras tutorial: https://github.com/fchollet/keras/blob/master/examples/imdb_bidirectional_lstm.py

# this is the placeholder tensor for the input sequences
sequence = Input(shape=(max_len,), dtype='int32')
# this embedding layer will transform the sequences of integers
# into vectors of size embedding
# embedding layer converts dense int input to one-hot in real time to save memory
embedded = Embedding(max_features, embedding_neurons, input_length=max_len)(sequence)
# normalize embeddings by input/word in sentence
bnorm = BatchNormalization()(embedded)

# apply forwards LSTM layer size lstm_neurons
forwards = GRU(lstm_neurons, dropout_W=0.4, dropout_U=0.4)(bnorm)
# apply backwards LSTM
backwards = GRU(lstm_neurons, dropout_W=0.4, dropout_U=0.4, go_backwards=True)(bnorm)

# concatenate the outputs of the 2 LSTMs
merged = merge([forwards, backwards], mode='concat', concat_axis=-1)
after_dp = Dropout(0.5)(merged)
output = Dense(1, activation='sigmoid')(after_dp)

model_bidir_atom = Model(input=sequence, output=output)
# review model structure
print(model_bidir_atom.summary())

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_2 (InputLayer)             (None, 200)           0                                            
____________________________________________________________________________________________________
embedding_2 (Embedding)          (None, 200, 128)      640000      input_2[0][0]                    
____________________________________________________________________________________________________
batchnormalization_2 (BatchNorma (None, 200, 128)      512         embedding_2[0][0]                
____________________________________________________________________________________________________
gru_2 (GRU)                      (None, 64)            37056       batchnormalization_2[0][0]       
___________________________________________________________________________________________

In [17]:

# Bi-directional Atom

# try using different optimizers and different optimizer configs
model_bidir_atom.compile('rmsprop', 'binary_crossentropy', metrics=['accuracy','precision', 'recall', 'fmeasure'])

print('Train...')
start_time = time.time()

history_bidir_atom = model_bidir_atom.fit(X_train, y_train,
                    batch_size=batch_size,
                    nb_epoch=epochs,
                    validation_data=[X_test, y_test], 
                    verbose=2)

end_time = time.time()
average_time_per_epoch = (end_time - start_time) / epochs
print("avg sec per epoch:", average_time_per_epoch)
scores = model_fdir_atom.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Train...
Train on 24 samples, validate on 28 samples
Epoch 1/2
0s - loss: 0.8105 - acc: 0.5417 - precision: 0.6000 - recall: 0.4615 - fmeasure: 0.5217 - val_loss: 0.6867 - val_acc: 0.5714 - val_precision: 0.5200 - val_recall: 1.0000 - val_fmeasure: 0.6842
Epoch 2/2
0s - loss: 0.6841 - acc: 0.6250 - precision: 0.7000 - recall: 0.5385 - fmeasure: 0.6087 - val_loss: 0.6830 - val_acc: 0.5000 - val_precision: 0.4815 - val_recall: 1.0000 - val_fmeasure: 0.6500
avg sec per epoch: 10.0065000057
Accuracy: 50.00%
